## Ridesharing driver behavior

In [185]:
import pandas as pd
import math
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.distributions as dist
import biogeme.results as res
import biogeme.optimization as opt
import biogeme.messaging as msg
from biogeme.expressions import bioDraws, Beta, DefineVariable, RandomVariable, log, RandomVariable, Integrate, Elem, bioNormalCdf, log, PanelLikelihoodTrajectory, MonteCarlo, exp

In [186]:
# Read the data
df_trip1 = pd.read_csv("WTRL.csv",sep=',',index_col = 0)

In [187]:
print(df_trip1.shape)
df_trip1 = df_trip1.dropna()
print(df_trip1.shape)

(1085, 74)
(1085, 74)


In [188]:
pd.set_option('display.max_columns', None)

In [189]:
database = db.Database("WTRL.csv",df_trip1)
database.panel("ID")

#from headers import *
# The following statement allows you to use the names of the variable
# as Python variable.
globals().update(database.variables)

In [190]:
## describe the data
pd.set_option('display.max_columns', None)
df_trip1.describe()

,ID,AGE,COMP_Uber,COMP_Lyft,WEEK.TRIPS,WEEK.TRIPS.RJ,GENDER,HISPANIC,RACE,IMMG,EDU,STDT,EMPL,ZIPCODE,HHSIZE,CHILD,HHINCOME,INDVINCOME,WKHOURS,TGINCOME,SOURCE,POVERTY,WT,RL,ADI,ADH,DMLS,DMLW,DMDS_1,DMDS_2,DMDS_3,DMDS_4,DMDW_1,DMDW_2,DMDW_3,DMDW_4,RTD_1,RTD_2,RTD_3,RTD_4,obs,DTGINCOME,DTGWKHOURS,ADI_TGINCOME,ADH_WKHOURS,INCOME_TG,WKHOURS_TG,ER,SURGERATE1,SURGERATE2,SURGERATE3,SURGERATE4,WTRATE1,WTRATE2,WTRATE3,WTRATE4,surgeRank3,surgeRank1,surgeRank2,surgeRank4,surgeRank5,RTDRank1,RTDRank2,RTDRank3,RTDRank4,WTRank3,WTRank1,WTRank2,WTRank4,WTRank5,RTD10_1,RTD10_2,RTD10_3,RTD10_4,_biogroups
count,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.00000,1085.000000,1085.000000,1085.0,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000
mean,93.073733,0.530876,0.640553,0.857143,53.133641,3.728111,0.046083,0.082949,2.464516,0.961290,1.500461,0.117972,1.692166,96680.506912,3.150230,1.058065,6.247926,5.81106,6.699539,156.165899,3.0,1.296774,0.514286,1.551152,182.679724,8.898341,4.258065,8.932719,4.626728,4.988018,4.232258,4.044240,7.894931,9.034101,8.905069,8.659908,14.305991,14.298618,13.697696,13.856221,3.475576,0.705069,0.727189,0.698986,0.724977,0.718894,0.717051,26.478093,0.537911,0.512243,0.473103,0.524670,0.764017,1.022281,1.034793,0.939424,2.297696,2.317051,2.205530,2.338249,2.435945,2.335484,2.239631,2.197235,2.155760,2.327189,2.213825,2.337327,2.305069,2.376037,0.666359,0.606452,0.572350,0.543779,91.846083
std,53.572588,0.581238,0.480060,0.350088,29.720618,5.146632,0.234670,0.362615,1.294996,0.243692,0.674557,0.392385,0.632152,11315.657224,2.047975,1.633184,2.028921,1.80310,4.445866,113.597050,0.0,0.974035,0.500026,1.712342,113.577677,4.236883,5.710719,11.308819,5.587762,5.901783,5.614086,5.849108,10.864140,11.488633,11.554269,11.123572,8.782867,8.981698,8.858713,9.617096,1.709158,0.456222,0.445610,0.575651,0.568234,0.449746,0.450640,23.774011,0.864152,0.721333,0.797975,0.997742,1.438317,1.846587,1.809328,1.555938,0.985524,1.064755,1.049146,1.042838,1.111330,1.100102,1.123260,1.101740,1.160813,1.603400,1.612449,1.598125,1.606037,1.634097,0.471730,0.488762,0.494966,0.498309,52.752584
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,9188.000000,1.000000,0.000000,1.000000,1.00000,0.000000,0.000000,3.0,0.000000,0.000000,0.000000,40.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000,4.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.272727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,46.000000,0.000000,0.000000,1.000000,35.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,0.000000,2.000000,98103.000000,1.000000,0.000000,5.000000,5.00000,0.000000,0.000000,3.0,1.000000,0.000000,0.000000,100.000000,5.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,8.000000,8.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.937500,0.000000,0

### Lower level: mode choice: MNL

In [191]:
INCOMETG_1a = DefineVariable('INCOMETG_1a',INCOME_TG == 1, database)
WKHOURSTG_1a = DefineVariable('WKHOURSTG_1a',WKHOURS_TG == 1, database)

FEMALE = DefineVariable('FEMALE',GENDER == 1, database)

AGE_MIDDLE = DefineVariable('AGE_MIDDLE',AGE == 1, database)
AGE_OLDER = DefineVariable('AGE_OLDER',AGE == 2, database)

EMPL_FT = DefineVariable('EMPL_FT',EMPL == 1, database)
EMPL_PT = DefineVariable('EMPL_PT',EMPL == 2, database)

INCOME_EXTLOW = DefineVariable('INCOME_EXTLOW',POVERTY == 0, database)
INCOME_VERYLOW = DefineVariable('INCOME_VERYLOW',POVERTY == 1, database)
INCOME_LOW = DefineVariable('INCOME_LOW',POVERTY == 2, database)

In [192]:
# Parameters to be estimated
## continue working
CONTINUEWK = Beta('CONTINUEWK', 1, None, None, 0)
CONTINUEWK_S = CONTINUEWK * bioDraws('CONTINUEWK_S', 'NORMAL')

## relocate to other neighborhoods
RELO = Beta('RELO', 1, None, None, 0)
RELO_S = RELO * bioDraws('RELO_S', 'NORMAL')

# one cost/waiting time
ASC_N3 = Beta('ASC_N3', 0.05, -1000, 1000, 0) ## Stay in the same place

B_DTGINCOME = Beta('B_DTGINCOME', 0.05, -1000, 1000, 0) ## dummy variable of income
B_DTGWKHOURS = Beta('B_DTGWKHOURS', 0.05, -1000, 1000, 0) ## dummy variable of working time

B_DSURGE = Beta('B_DSURGE', 0.05, -1000, 1000, 0) ## Surge price in the same place
B_DWT = Beta('B_DWT', 0.05, -1000, 1000, 0) ## Waiting time in the same place

B_ADI_TG = Beta('B_ADI_TG', 0.05, -1000, 1000, 0) ## Accumulated daily income target rate
B_ADI_TG_1a = Beta('B_ADI_TG_1a', 0.05, -1000, 1000, 0) ## Accumulated daily income target rate
B_ADI_TG_075a = Beta('B_ADI_TG_075a', 0.05, -1000, 1000, 0) ## Accumulated daily income target rate
B_ADI_TG_05a = Beta('B_ADI_TG_05a', 0.05, -1000, 1000, 0) ## Accumulated daily income target rate
B_ADI_TG_025a = Beta('B_ADI_TG_025a', 0.05, -1000, 1000, 0) ## Accumulated daily income target rate
B_ADI_TG_000a = Beta('B_ADI_TG_000a', 0.05, -1000, 1000, 0) ## Accumulated daily income target rate

B_ADH_TG = Beta('B_ADH_TG', 0.05, -1000, 1000, 0) ## Accumulated daily working hours target rate
B_ADH_TG_1a = Beta('B_ADH_TG_1a', 0.05, -1000, 1000, 0) ## Accumulated daily working hours target rate
B_ADH_TG_075a = Beta('B_ADH_TG_075a', 0.05, -1000, 1000, 0) ## Accumulated daily working hours target rate
B_ADH_TG_05a = Beta('B_ADH_TG_05a', 0.05, -1000, 1000, 0) ## Accumulated daily working hours target rate
B_ADH_TG_025a = Beta('B_ADH_TG_025a', 0.05, -1000, 1000, 0) ## Accumulated daily working hours target rate
B_ADH_TG_000a = Beta('B_ADH_TG_000a', 0.05, -1000, 1000, 0) ## Accumulated daily working hours target rate

B_ADIH = Beta('B_ADIH', 0.05, -1000, 1000, 0) ## Accumulated daily income/working hours
B_SURGE_RATE = Beta('B_SURGE_RATE', 0.05, -1000, 1000, 0) ## Surge price rate
B_SURGE_R = Beta('B_SURGE_R', 0.05, -1000, 1000, 0) ## Surge price rank
B_WT_RATE = Beta('B_WT_RATE', 0.05, -1000, 1000, 0)
B_WT_R = Beta('B_WT_R', 0.05, -1000, 1000, 0) ## Waiting time
B_RLTIME_R = Beta('B_RLTIME_R', 0.05, -1000, 1000, 0) ## Relocation time rank
B_RLTIME_10min = Beta('B_RLTIME_10min', 0.05, -1000, 1000, 0) ## Relocation time 10 minutes

B_SURGE_STAY = Beta('B_SURGE_STAY', 0.05, -1000, 1000, 0) ## Surge price rate
B_WT_STAY = Beta('B_WT_STAY', 0.05, -1000, 1000, 0) ## Waiting time

# Interactions
B_TGINCOME_ADIH = Beta('B_TGINCOME_ADIH', 0.05, -1000, 1000, 0) ## Interactions between earning rate and whether have target
B_TGWKHOURS_ADIH = Beta('B_TGWKHOURS_ADIH', 0.05, -1000, 1000, 0) ## Interactions between earning rate and whether have target
B_TGINCOME_ADI_TGINCOME = Beta('B_TGINCOME_ADI_TGINCOME', 0.05, -1000, 1000, 0)
B_TGWKHOURS_ADH_WKHOURS = Beta('B_TGWKHOURS_ADH_WKHOURS', 0.05, -1000, 1000, 0)
B_TGINCOME_ADI_TGINCOME1a = Beta('B_TGINCOME_ADI_TGINCOME1a', 0.05, -1000, 1000, 0)
B_TGWKHOURS_ADH_WKHOURS1a = Beta('B_TGWKHOURS_ADH_WKHOURS1a', 0.05, -1000, 1000, 0)
B_TGINCOME_SURGE = Beta('B_TGINCOME_SURGE', 0.05, -1000, 1000, 0) ## Interactions between surge price and whether have target
B_TGINCOME_WT = Beta('B_TGINCOME_WT', 0.05, -1000, 1000, 0) ## Interactions between waiting time and whether have target
B_TGWKHOURS_SURGE = Beta('B_TGWKHOURS_SURGE', 0.05, -1000, 1000, 0) ## Interactions between waiting time and whether have target
B_TGWKHOURS_WT = Beta('B_TGWKHOURS_WT', 0.05, -1000, 1000, 0) ## Interactions between waiting time and whether have target

## Socio
B_AGE_MIDDLE = Beta('B_AGE_MIDDLE', 0.05, -1000, 1000, 0)
B_AGE_OLDER = Beta('B_AGE_OLDER', 0.05, -1000, 1000, 0)
B_GENDER = Beta('B_GENDER', 0.05, -1000, 1000, 0)
B_GENDER1 = Beta('B_GENDER1', 0.05, -1000, 1000, 0)
B_EDU = Beta('B_EDU', 0.05, -1000, 1000, 0)
B_CHILD = Beta('B_CHILD', 0.05, -1000, 1000, 0)
B_COMP_Uber_RL = Beta('B_COMP_Uber_RL', 0.05, -1000, 1000, 0)
B_COMP_Lyft_RL = Beta('B_COMP_Lyft_RL', 0.05, -1000, 1000, 0)
B_COMP_Uber_WT = Beta('B_COMP_Uber_WT', 0.05, -1000, 1000, 0)
B_COMP_Lyft_WT = Beta('B_COMP_Lyft_WT', 0.05, -1000, 1000, 0)
B_IMMG = Beta('B_IMMG', 0.05, -1000, 1000, 0)
B_EMPL_FT = Beta('B_EMPL_FT', 0.05, -1000, 1000, 0)
B_EMPL_PT = Beta('B_EMPL_PT', 0.05, -1000, 1000, 0)
B_INCOME_EXTLOW = Beta('B_INCOME_EXTLOW', 0.05, -1000, 1000, 0)
B_INCOME_VERYLOW = Beta('B_INCOME_VERYLOW', 0.05, -1000, 1000, 0)
B_INCOME_LOW = Beta('B_INCOME_LOW', 0.05, -1000, 1000, 0)

## definition of the utility functions
         ## earning rate main effect
STOPWK = B_ADIH*ER + \
         B_ADI_TG*ADI_TGINCOME + B_ADH_TG*ADH_WKHOURS + \
         B_ADI_TG_1a*INCOMETG_1a + B_ADH_TG_1a*WKHOURSTG_1a + \
         B_DTGINCOME*DTGINCOME + B_DTGWKHOURS*DTGWKHOURS + \
         B_TGINCOME_ADIH*ER*DTGINCOME + B_TGWKHOURS_ADIH*ER*DTGWKHOURS + \
         B_TGINCOME_ADI_TGINCOME1a*INCOMETG_1a*DTGINCOME + B_TGWKHOURS_ADH_WKHOURS1a*WKHOURSTG_1a*DTGWKHOURS + \
         B_TGINCOME_ADI_TGINCOME*ADI_TGINCOME*DTGINCOME + B_TGWKHOURS_ADH_WKHOURS*ADH_WKHOURS*DTGWKHOURS + \
         B_DSURGE*DMLS + B_DWT*DMLW + \
         B_TGINCOME_SURGE*DMLS*DTGINCOME + B_TGINCOME_WT*DMLW*DTGINCOME + B_TGWKHOURS_SURGE*DMLS*DTGWKHOURS + B_TGWKHOURS_WT*DMLW*DTGWKHOURS +  \
         B_GENDER*FEMALE + B_AGE_MIDDLE*AGE_MIDDLE + B_AGE_OLDER*AGE_OLDER + B_INCOME_EXTLOW*INCOME_EXTLOW + B_INCOME_VERYLOW*INCOME_VERYLOW + B_INCOME_LOW*INCOME_LOW + B_EMPL_FT*EMPL_FT + B_EMPL_PT*EMPL_PT 

N3 = ASC_N3 + B_SURGE_STAY*DMLS + B_WT_STAY*DMLW + B_SURGE_R*surgeRank3 + B_WT_R*WTRank3 + CONTINUEWK_S
N1 = B_SURGE_R*surgeRank1 + B_WT_R*WTRank1 + B_RLTIME_R*RTDRank1 + B_SURGE_RATE*SURGERATE1 + B_WT_RATE*WTRATE1 + B_RLTIME_10min*RTD10_1 + CONTINUEWK_S + RELO_S
N2 = B_SURGE_R*surgeRank2 + B_WT_R*WTRank2 + B_RLTIME_R*RTDRank2 + B_SURGE_RATE*SURGERATE2 + B_WT_RATE*WTRATE2 + B_RLTIME_10min*RTD10_2 + CONTINUEWK_S + RELO_S
N4 = B_SURGE_R*surgeRank4 + B_WT_R*WTRank4 + B_RLTIME_R*RTDRank3 + B_SURGE_RATE*SURGERATE3 + B_WT_RATE*WTRATE3 + B_RLTIME_10min*RTD10_3 + CONTINUEWK_S + RELO_S
N5 = B_SURGE_R*surgeRank5 + B_WT_R*WTRank5 + B_RLTIME_R*RTDRank4 + B_SURGE_RATE*SURGERATE4 + B_WT_RATE*WTRATE4 + B_RLTIME_10min*RTD10_4 + CONTINUEWK_S + RELO_S

In [193]:
## Associate the utility functions with the numbering of the alternatives
V_mode_1 = {
    0:STOPWK,
    1:N1,
    2:N2,
    3:N3,
    4:N4,
    5:N5
}

## Availability of each alternative, as described in the dataset
av_mode_1 = {
    0:1,
    1:1,
    2:1,
    3:1,
    4:1,
    5:1
}

# Definition of the model
obsprob = models.logit(V_mode_1, av_mode_1, RL)

# Conditional to the random parameters, the likelihood of all observations for
# one individual (the trajectory) is the product of the likelihood of
# each observation.
condprobIndiv = PanelLikelihoodTrajectory(obsprob)

# We integrate over the random parameters using Monte-Carlo
logprob = log(MonteCarlo(condprobIndiv))

# Define level of verbosity
logger = msg.bioMessage()
#logger.setSilent()
#logger.setWarning()
#logger.setGeneral()
logger.setDetailed()
#logger.setDebug()

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, numberOfDraws=100)
biogeme.modelName = 'RL-ML'

# Estimate the parameters.
results = biogeme.estimate()
modeResults = results.getEstimatedParameters()
modeResults

[22:46:48] < General >   Remove 40 unused variables from the database as only 45 are used.
[22:46:48] < Detailed >  It is suggested to scale the following variables.
[22:46:48] < Detailed >  Multiply ID by	0.01 because the largest (abs) value is	186.0
[22:46:48] < Detailed >  Multiply ER by	0.01 because the largest (abs) value is	150.0
[22:46:48] < Detailed >  To remove this feature, set the parameter suggestScales to False when creating the BIOGEME object.
[22:46:49] < Detailed >  Log likelihood (N = 184):  -3189.691
[22:46:49] < Detailed >  ** Optimization: BFGS with trust region for simple bounds
[22:46:49] < General >   Log likelihood (N = 184):  -3189.691 Gradient norm:      2e+04  
[22:46:50] < Detailed >  Log likelihood (N = 184): -1.797693e+308
[22:46:50] < Detailed >  1 f=  17.33528 projected rel. grad.=   4.5 delta=   0.5 rho=-3.5e+303 -
[22:46:51] < Detailed >  Log likelihood (N = 184):  -25383.91
[22:46:51] < Detailed >  2 f=  17.33528 projected rel. grad.=   4.5 delta=  0.

[22:47:39] < Detailed >  Log likelihood (N = 184):  -1054.966
[22:47:39] < Detailed >  36 f=  5.677193 projected rel. grad.= 0.047 delta= 0.031 rho=  -1.4 -
[22:47:40] < Detailed >  Log likelihood (N = 184):   -1043.51
[22:47:41] < General >   Log likelihood (N = 184):   -1043.51 Gradient norm:      2e+02  
[22:47:41] < Detailed >  37 f=  5.671251 projected rel. grad.= 0.035 delta= 0.031 rho=  0.26 +
[22:47:41] < Detailed >  Log likelihood (N = 184):  -1039.847
[22:47:42] < General >   Log likelihood (N = 184):  -1039.847 Gradient norm:      8e+01  
[22:47:42] < Detailed >  38 f=  5.651343 projected rel. grad.= 0.017 delta= 0.031 rho=  0.79 +
[22:47:43] < Detailed >  Log likelihood (N = 184):  -1046.215
[22:47:43] < Detailed >  39 f=  5.651343 projected rel. grad.= 0.017 delta= 0.016 rho=  -2.5 -
[22:47:44] < Detailed >  Log likelihood (N = 184):  -1040.263
[22:47:44] < Detailed >  40 f=  5.651343 projected rel. grad.= 0.017 delta=0.0078 rho= -0.31 -
[22:47:44] < Detailed >  Log likeli

[22:48:36] < General >   Log likelihood (N = 184):  -983.4254 Gradient norm:      6e+01  
[22:48:36] < Detailed >  73 f=  5.344703 projected rel. grad.=  0.01 delta=     1 rho=   1.4 ++
[22:48:37] < Detailed >  Log likelihood (N = 184):   -980.248
[22:48:38] < General >   Log likelihood (N = 184):   -980.248 Gradient norm:      2e+02  
[22:48:38] < Detailed >  74 f=  5.327435 projected rel. grad.=  0.03 delta=     2 rho=   1.2 ++
[22:48:39] < Detailed >  Log likelihood (N = 184):  -978.6368
[22:48:40] < General >   Log likelihood (N = 184):  -978.6368 Gradient norm:      5e+01  
[22:48:40] < Detailed >  75 f=  5.318679 projected rel. grad.=0.0075 delta=     4 rho=   1.3 ++
[22:48:41] < Detailed >  Log likelihood (N = 184):  -977.3724
[22:48:42] < General >   Log likelihood (N = 184):  -977.3724 Gradient norm:      6e+01  
[22:48:42] < Detailed >  76 f=  5.311806 projected rel. grad.= 0.011 delta=     8 rho=   1.4 ++
[22:48:42] < Detailed >  Log likelihood (N = 184):  -976.4312
[22:48:4

[22:49:35] < Detailed >  Log likelihood (N = 184):  -973.5119
[22:49:36] < General >   Log likelihood (N = 184):  -973.5119 Gradient norm:          3  
[22:49:36] < Detailed >  107 f=  5.290825 projected rel. grad.=0.00058 delta=1.7e+10 rho=   1.5 ++
[22:49:37] < Detailed >  Log likelihood (N = 184):  -973.5089
[22:49:38] < General >   Log likelihood (N = 184):  -973.5089 Gradient norm:          2  
[22:49:38] < Detailed >  108 f=  5.290809 projected rel. grad.=0.00034 delta=3.4e+10 rho=   1.4 ++
[22:49:39] < Detailed >  Log likelihood (N = 184):  -973.5071
[22:49:40] < General >   Log likelihood (N = 184):  -973.5071 Gradient norm:        0.6  
[22:49:40] < Detailed >  109 f=  5.290799 projected rel. grad.=8.9e-05 delta=6.9e+10 rho=   1.3 ++
[22:49:41] < Detailed >  Log likelihood (N = 184):  -973.5062
[22:49:42] < General >   Log likelihood (N = 184):  -973.5062 Gradient norm:          1  
[22:49:42] < Detailed >  110 f=  5.290795 projected rel. grad.=0.00022 delta=1.4e+11 rho=   1.5

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_N3,1.176889,0.314568,3.741282,1.830837e-04,0.311441,3.778847,1.575564e-04
B_ADH_TG,2.353065,0.315899,7.448786,9.414691e-14,0.415889,5.657919,1.532193e-08
B_ADH_TG_1a,2.485085,0.482487,5.150579,2.596839e-07,0.528195,4.704859,2.540415e-06
B_ADIH,-0.190892,0.031631,-6.034910,1.590513e-09,0.042361,-4.506302,6.596724e-06
B_ADI_TG,0.267736,0.308044,0.869147,3.847667e-01,0.344035,0.778222,4.364379e-01
B_ADI_TG_1a,0.964684,0.519770,1.855981,6.345623e-02,0.549881,1.754349,7.937077e-02
B_AGE_MIDDLE,0.021296,0.318432,0.066877,9.466793e-01,0.297682,0.071539,9.429688e-01
B_AGE_OLDER,0.443026,0.772558,0.573454,5.663377e-01,0.899147,0.492719,6.222115e-01
B_DSURGE,0.026988,0.050825,0.530993,5.954237e-01,0.048891,0.552000,5.809481e-01
B_DTGINCOME,-1.311607,0.569008,-2.305077,2.116225e-02,0.597628,-2.194687,2.818605e-02


### Simulation

In [9]:
# The choice model is a logit, with availability conditions
prob_Walking1 = models.logit(V_mode_1, av_mode_1, 1)
prob_Biking1 = models.logit(V_mode_1, av_mode_1, 2)
prob_Transit1 = models.logit(V_mode_1, av_mode_1, 3)
prob_Ridehailing1 = models.logit(V_mode_1, av_mode_1, 4)
prob_SD_ridehailing1 = models.logit(V_mode_1, av_mode_1, 5)
prob_Car1 = models.logit(V_mode_1, av_mode_1, 6)
prob_SD_car1 = models.logit(V_mode_1, av_mode_1, 7)

simulate = {'Walking_1': prob_Walking1,
            'Biking_1': prob_Biking1,
            'Transit_1': prob_Transit1,
            'Ridehailing_1': prob_Ridehailing1,
            'SD_ridehailing_1': prob_SD_ridehailing1,
            'Car_1': prob_Car1,
            'SD_car_1': prob_SD_car1}
                      
biogeme = bio.BIOGEME(database, simulate)
SV_trip1 = biogeme.simulate(results_trip1.getBetaValues())

##print(simulatedValues)
pd.set_option('display.max_rows', None)
SV_trip1

,Walking_1,Biking_1,Transit_1,Ridehailing_1,SD_ridehailing_1,Car_1,SD_car_1
1,1.387470e-13,2.296248e-06,0.000000,0.543934,4.560642e-01,0.000000,0.000000
2,1.593685e-14,2.637532e-07,0.000000,0.061733,5.929384e-02,0.878973,0.000000
3,2.173301e-05,9.639887e-04,0.014894,0.015246,7.605210e-03,0.000000,0.961270
4,1.386105e-04,6.148204e-03,0.077535,0.068750,1.095522e-01,0.737876,0.000000
5,5.805650e-07,2.649104e-02,0.082979,0.086097,7.643611e-02,0.727997,0.000000
6,5.107952e-07,9.641530e-02,0.087052,0.132683,7.715070e-02,0.606698,0.000000
7,4.236444e-06,1.124063e-01,0.094244,0.101694,1.432460e-01,0.548405,0.000000
8,5.099029e-04,3.365505e-02,0.102304,0.147467,1.071210e-01,0.608943,0.000000
9,3.993074e-04,2.635544e-02,0.080115,0.108883,7.576171e-02,0.708486,0.000000
10,1.491044e-01,4.470880e-01,0.104141,0.145404,1.542619e-01,0.000000,0.000000
